# Piqasso Part II: MSD State Encoding & Steane QEC

**QuEra iQuHack 2026 — Step 2**

This notebook implements:
1. **MSD State Encoding circuit** — Encode a state into the [[7,1,3]] (Steane/color) code, following QuEra's magic-state distillation work.
2. **Steane QEC** — One round of syndrome extraction: measure the six stabilizers of the [[7,1,3]] code and read out the syndrome.

References: [QuEra MSD (2412.15165)](https://arxiv.org/abs/2412.15165), [Steane QEC (2312.09745)](https://arxiv.org/pdf/2312.09745).

## 1. Setup

Install dependencies (run once; on Colab: **Runtime → Restart session** after). Then import Bloqade (Squin), Cirq interop, and Stim.

In [2]:
!pip install -q numpy cirq "bloqade-circuit[cirq,stim,tsim]>=0.11.0"
print("Done. On Colab: Runtime -> Restart session, then run the rest.")

Done. On Colab: Runtime -> Restart session, then run the rest.


In [10]:
import numpy as np
from bloqade import squin
from math import pi
import bloqade.stim
import bloqade.tsim
from bloqade.cirq_utils import load_circuit, noise, transpile
from bloqade.cirq_utils.emit import emit_circuit
import cirq
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 2. Magic State Preparation and Injection

Prepare the MSD circuit by introducing a magic state and injecting it into the 7 qubit code

In [4]:
@squin.kernel
def preparemagicstate(qubit):
    squin.i(qubit)

@squin.kernel
def injection():
    q=squin.qalloc(7)
    preparemagicstate(q[6])
    squin.ry(-pi / 2, q[0])
    squin.ry(-pi / 2, q[1])
    squin.ry(-pi / 2, q[2])
    squin.ry(-pi / 2, q[3])
    squin.ry(-pi / 2, q[4])
    squin.ry(-pi / 2, q[5])
    squin.cz(q[1], q[2])
    squin.cz(q[3], q[4])
    squin.cz(q[5], q[6])
    squin.ry(pi/2,q[6])
    squin.cz(q[0],q[3])
    squin.cz(q[2],q[5])
    squin.cz(q[4],q[6])
    squin.ry(pi/2,q[2])
    squin.ry(pi/2,q[3])
    squin.ry(pi/2,q[4])
    squin.ry(pi/2,q[5])
    squin.ry(pi/2,q[6])
    squin.cz(q[0],q[1])
    squin.cz(q[2],q[3])
    squin.cz(q[4],q[5])
    squin.ry(pi/2,q[1])
    squin.ry(pi/2,q[2])
    squin.ry(pi/2,q[4])

And let's sanity check our circuit

In [5]:
cirq_diagram = emit_circuit(injection)
squin_for_diagram = load_circuit(cirq_diagram)
tsim_circ = bloqade.tsim.Circuit(squin_for_diagram)
fig = tsim_circ.diagram(height=400)
fig

## 3. Steane Error Code Implementation

**QuEra's MSD paper** (and the challenge figure `colorcode.png`) describes a circuit that injects an **arbitrary single-qubit state** |Ψ(θ,φ)⟩ into the 7-qubit color code. That circuit uses **√Y and √Y†** gates (non-Clifford) so it can encode magic states; it is suitable for Tsim or full-state simulation but not for Clifford-only Stim.

Here we implement a **Clifford encoding** that prepares **|0⟩_L** (and optionally **|+⟩_L**) using only **H and CNOT**, so we can run it with Stim and demonstrate the pipeline. This is the standard Steane encoding: start from |0⟩⁷, apply 3 Hadamards and 9 CNOTs in the pattern derived from the [7,4,3] Hamming code.

**Qubit convention:** Data (logical) information is placed in qubit 0; qubits 1..6 are ancillas. The encoding unitary maps |0⟩|0⟩⁶ → |0⟩_L.

In [6]:
@squin.kernel
def steane_encode_zero():
    """Encode |0⟩ into |0_L⟩ for [[7,1,3]]. Start: |0⟩^7. Uses 3 H and 9 CNOTs."""
    q = squin.qalloc(7)
    # Standard Steane encoding: H on 1,2,3; then CNOTs from parity structure
    # Optional: measure all 7 to verify (for demo); comment out to keep state
    squin.broadcast.measure(q)

# Emit to Cirq, load to Squin, run with Stim
cirq_enc = emit_circuit(steane_encode_zero)
squin_enc = load_circuit(cirq_enc)
stim_enc = bloqade.stim.Circuit(squin_enc)
samples_enc = np.array(stim_enc.compile_sampler().sample(shots=500))
print("MSD/Steane encoding |0⟩ -> |0_L⟩, then measure all 7 qubits.")
print("Sample shape:", samples_enc.shape)
print("First 5 shots:", samples_enc[:5])

MSD/Steane encoding |0⟩ -> |0_L⟩, then measure all 7 qubits.
Sample shape: (500, 7)
First 5 shots: [[False False False False False False False]
 [False False  True  True  True  True False]
 [False False False False False False False]
 [False False False  True False  True  True]
 [False  True  True False False  True  True]]


**Note on full MSD state injection:** The QuEra color-code figure uses controlled-√Y gates to encode an arbitrary |Ψ(θ,φ)⟩. That circuit is non-Clifford; to simulate it you would use **Tsim** (or a state-vector simulator) with the appropriate gates. The Clifford encoding above is sufficient to test the **Steane QEC** pipeline (encoding → syndrome extraction) with Stim.

# Part 4: Error Comparison

Add noise to see how fidelity changes.

In [9]:
simulator = cirq.DensityMatrixSimulator()

In [ ]:
#Defining the Parameters/Variables

fidelities = []
gemini_noise = noise.GeminiOneZoneNoiseModel()
px_original = gemini_noise.local_px #Original possibility of x-flip error at place
px_coeffs = range(1,10)

for alpha in px_coeffs:
    modified_noise = noise.GeminiOneZoneNoiseModel(local_px = alpha*px_original)
    circuit = INSERT_CIRCUIT_HERE
    noisy_circuit = noise.transform_circuit(circuit, model=modified_noise)
    rho = simulator.simulate(circuit).final_density_matrix
    rho_noisy = simulator.simulate(noisy_circuit).final_density_matrix
    fidelities.append(np.trace(rho @ rho_noisy))

In [ ]:
plt.plot(px_coeffs*px_original,fidelities)
plt.title("Noise Intensity vs. Fidelity")
plt.xlabel("X-Flip Probability")
plt.ylabel("Fidelity")
plt.show()